In [1]:
import os
import tensorflow as tf
import keras
import cv2
import keras.preprocessing.image
import keras_retinanet
import math
from keras_retinanet import models, losses
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.models.resnet import resnet50_retinanet
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from shared import get_session

#Paths GTSDB
BASE_DIR = os.getcwd()
SNAPSHOTS_DIR = os.path.join(BASE_DIR, 'snapshots')
LOGS_DIR = os.path.join(BASE_DIR, 'logs')
DATASET_DIR = os.path.join(BASE_DIR, 'datasets', 'GTSDB', 'FullIJCNN2013')
GROUND_TRUTH = os.path.join(DATASET_DIR, "gt.txt")
TRAIN_GROUND_TRUTH = os.path.join(DATASET_DIR, "gt_train.csv")
TEST_GROUND_TRUTH = os.path.join(DATASET_DIR, "gt_test.csv")
CLASS_MAPPINGS = os.path.join(DATASET_DIR, "cm.csv")

#Create folders
if not os.path.exists(SNAPSHOTS_DIR):
    os.makedirs(SNAPSHOTS_DIR)
    
if not os.path.exists(LOGS_DIR):
    os.makedirs(LOGS_DIR)

/home/madspoderpetersen/deep-traffic/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_models(num_classes, weights):

    model = models.resnet.resnet_retinanet(num_classes)
    if weights == "imagenet":
        backbone = models.backbone('resnet50')
        imgnet_weights = backbone.download_imagenet()
        #Load by_name only inputs into the resnet backbone (Tak Thomas)
        model.load_weights(imgnet_weights, skip_mismatch=True, by_name=True)
    elif weights:
        model.load_weights(weights, skip_mismatch=True)

    training_model = model
    prediction_model = retinanet_bbox(model=model)

    model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

    return model, training_model, prediction_model

def create_train_generator(batch_size):
    train_generator = CSVGenerator(
        TRAIN_GROUND_TRUTH,
        CLASS_MAPPINGS,
        batch_size=batch_size,
    )
    return train_generator

def create_validation_generator(batch_size):
    validation_generator = CSVGenerator(
        TEST_GROUND_TRUTH,
        CLASS_MAPPINGS,
        batch_size=batch_size,
    )
    return validation_generator

def create_callbacks(model, training_model, prediction_model, batch_size):
    callbacks = []

    # save the prediction model
    checkpoint = keras.callbacks.ModelCheckpoint(
        os.path.join(SNAPSHOTS_DIR, 'custom_resnet50_{epoch:02d}.h5'),
        verbose=1,  monitor='val_acc', save_best_only=True, save_weights_only=False, mode='auto', period=1)
                      
    checkpoint = RedirectModel(checkpoint, prediction_model)
    callbacks.append(checkpoint)

    
    evaluation = Evaluate(validation_generator)
    evaluation = RedirectModel(evaluation, prediction_model)
    callbacks.append(evaluation)
    
    
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1,
                                                     mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
    
    early_stop = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
    #callbacks.append(early_stop)
    
    tensorboard_callb = keras.callbacks.TensorBoard(log_dir=LOGS_DIR, histogram_freq=0, batch_size=batch_size, 
                                                    write_graph=True, write_grads=False, write_images=True, 
                                                    embeddings_freq=0, embeddings_layer_names=None, 
                                                    embeddings_metadata=None)
    callbacks.append(tensorboard_callb)

    return callbacks

In [3]:
batch_size = 2

keras.backend.tensorflow_backend.set_session(get_session())
train_generator = create_train_generator(batch_size)
validation_generator = create_validation_generator(batch_size)

#From start
#model, training_model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights=None)
#From imagenet
model, training_model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights="imagenet")
#From earlier 
#snapshot = os.path.join(SNAPSHOTS_DIR, "custom_resnet50_10.h5")
#model, training_model, prediction_model = create_models(num_classes=train_generator.num_classes(), weights=snapshot)

callbacks = create_callbacks(model, training_model, prediction_model, batch_size)
print(train_generator.size())
# start training
training_model.fit_generator(
    generator = train_generator,
    #steps_per_epoch = 850,
    steps_per_epoch = math.floor(train_generator.size()/batch_size),
    epochs = 1000,
    verbose = 1,
    #validation_data=validation_generator,
    #validation_steps=math.floor(validation_generator.size()/batch_size),
    callbacks = callbacks,
)

Instructions for updating:
Use the retry module or similar alternatives.
604
Epoch 1/1000
302/302 [==============================] - 209s 691ms/step - loss: 5.1388 - regression_loss: 3.5897 - classification_loss: 1.5491


/home/madspoderpetersen/deep-traffic/venv/lib/python3.5/site-packages/keras/callbacks.py:435: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


0 0.0000
1 0.0000
2 0.0000
3 0.0000
4 0.0000
5 0.0000
6 0.0000
7 0.0000
8 0.0000
9 0.0000
10 0.0000
11 0.0000
12 0.0000
13 0.0000
14 0.0000
15 0.0000
16 0.0000
17 0.0000
18 0.0000
19 0.0000
20 0.0000
21 0.0000
22 0.0000
23 0.0000
24 0.0000
25 0.0000
26 0.0000
27 0.0000
28 0.0000
29 0.0000
30 0.0000
31 0.0000
32 0.0000
33 0.0000
34 0.0000
35 0.0000
36 0.0000
37 0.0000
38 0.0000
39 0.0000
40 0.0000
41 0.0000
42 0.0000
mAP: 0.0000
Epoch 2/1000
302/302 [==============================] - 163s 541ms/step - loss: 4.4906 - regression_loss: 3.0870 - classification_loss: 1.4036
0 0.0000
1 0.0000
2 0.0000
3 0.0000
4 0.0000
5 0.0000
6 0.0000
7 0.0000
8 0.0000
9 0.0000
10 0.0000
11 0.0000
12 0.0000
13 0.0000
14 0.0000
15 0.0000
16 0.0000
17 0.0000
18 0.0000
19 0.0000
20 0.0000
21 0.0000
22 0.0000
23 0.0000
24 0.0000
25 0.0000
26 0.0000
27 0.0000
28 0.0000
29 0.0000
30 0.0000
31 0.0000
32 0.0000
33 0.0000
34 0.0000
35 0.0000
36 0.0000
37 0.0000
38 0.0000
39 0.0000
40 0.0000
41 0.0000
42 0.0000
mAP: 

KeyboardInterrupt: 